<a href="https://colab.research.google.com/github/fabianoborgesbsb/APC_study/blob/main/APC_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt

In [4]:
import os

file_path = '/content/drive/My Drive/APC_study.ipynb'
print(os.path.exists(file_path))  # Deve retornar: True


True


In [5]:
import nbformat

with open(file_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# Salvar como novo notebook (opcional)
with open('/content/drive/My Drive/APC_study_fixed.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)



# Average APC each country

In [ ]:
%%bigquery --project=insyspo

SELECT countries.country_code,
  ANY_VALUE(countries.country) AS country,
  AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd
FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
JOIN (
  SELECT DISTINCT apcs.id, authorships.institution_id
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
  ON apcs.id = authorships.work_id
) AS institutions
ON apcs.id = institutions.id
JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
ON institutions.institution_id = countries.id
GROUP BY countries.country_code
ORDER BY avg_value_usd DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,country_code,country,avg_value_usd
0,CK,Cook Islands,5045.000000
1,IM,Isle of Man,3726.630290
2,AX,Aland Islands,3666.062500
3,GB,United Kingdom,3622.919975
4,US,United States,3564.686030
...,...,...,...
222,NI,Nicaragua,1001.088838
223,TJ,Tajikistan,964.795914
224,SV,El Salvador,868.427632
225,ID,Indonesia,514.570688


# Average APC every year each country

In [7]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs;
CREATE TABLE userdb_luis_fabiano_borges.apcs AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id, works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
    AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
)

Query is running:   0%|          |

""


In [8]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

Query is running:   0%|          |

Downloading:   0%|          |

# Each field

In [9]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS userdb_luis_fabiano_borges.apcs_engineering;
CREATE TABLE userdb_luis_fabiano_borges.apcs_engineering AS (
  SELECT countries.country_code,
    institutions.publication_year,
    ANY_VALUE(countries.country) AS country,
    AVG(CAST(apcs.value_usd AS FLOAT64)) AS avg_value_usd,
    COUNT(DISTINCT apcs.id) AS works
  FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
  JOIN (
    SELECT DISTINCT apcs.id, authorships.institution_id,
      works.publication_year
    FROM `insyspo.publicdb_openalex_2024_04_rm.works_apc` AS apcs
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS authorships
    ON apcs.id = authorships.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS works
    ON works.id = apcs.id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.works_topics` AS works_topics
    ON works.id = works_topics.work_id
    JOIN `insyspo.publicdb_openalex_2024_04_rm.topics` AS topics
    ON works_topics.topic_id = topics.id
    WHERE topics.field = 22
  ) AS institutions
  ON apcs.id = institutions.id
  JOIN `insyspo.publicdb_openalex_2024_04_rm.institutions` AS countries
  ON institutions.institution_id = countries.id
  WHERE institutions.publication_year >= 1990
  AND institutions.publication_year <= 2024
  GROUP BY countries.country_code, institutions.publication_year
  HAVING AVG(CAST(apcs.value_usd AS FLOAT64)) > 0.0
  ORDER BY institutions.publication_year, countries.country_code
);

Query is running:   0%|          |

""


# Correct tables for missing country codes

Aggregate

In [10]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs

Query is running:   0%|          |

Downloading:   0%|          |

In [11]:
df.columns

Index(['country_code', 'publication_year', 'country', 'avg_value_usd',
       'works'],
      dtype='object')

In [12]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

In [13]:
df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df[df['country']=='Netherlands']

/tmp/ipython-input-807249710.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)


,country_code,publication_year,country,avg_value_usd,works
128,NL,1990,Netherlands,3652.344606,6692
319,NL,1991,Netherlands,3650.152591,6913
509,NL,1992,Netherlands,3664.445768,7692
710,NL,1993,Netherlands,3708.915442,8264
914,NL,1994,Netherlands,3678.380257,8918
1117,NL,1995,Netherlands,3716.301863,9537
1324,NL,1996,Netherlands,3722.898974,10130
1527,NL,1997,Netherlands,3716.978325,10815
1733,NL,1998,Netherlands,3743.393525,10937
1938,NL,1999,Netherlands,3820.963455,10789


In [14]:
df.drop(row_drop,axis=0, inplace=True)

In [15]:
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs',if_exists ='replace')

/tmp/ipython-input-1159636301.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs',if_exists ='replace')
100%|██████████| 1/1 [00:00<00:00, 8019.70it/s]


Computer Science

In [16]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_computer_science

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_computer_science',if_exists ='replace')


/tmp/ipython-input-1790584851.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
/tmp/ipython-input-1790584851.py:22: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_computer_science',if_exists ='replace')
100%|██████████| 1/1 [00:00<

Medicine


In [18]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_medicine

Query is running:   0%|          |

Downloading:   0%|          |

In [19]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_medicine',if_exists ='replace')


/tmp/ipython-input-3848568122.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
/tmp/ipython-input-3848568122.py:22: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_medicine',if_exists ='replace')
100%|██████████| 1/1 [00:00<00:00, 2

Engineering

In [20]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_engineering

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_engineering',if_exists ='replace')


/tmp/ipython-input-1340294436.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
/tmp/ipython-input-1340294436.py:22: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_engineering',if_exists ='replace')
100%|██████████| 1/1 [00:00<00:00

Physics

In [22]:
%%bigquery df --project=insyspo

SELECT *
FROM userdb_luis_fabiano_borges.apcs_physics

Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
row_drop = []
for i, cc in enumerate(df['country_code']):
  if cc == None:
    if df.loc[i,'country'] == 'France':
      ccode = 'FR'
    elif df.loc[i,'country'] == 'India':
      ccode = 'IN'
    elif df.loc[i,'country'][:6] == 'United':
      ccode = 'US'
    elif df.loc[i,'country'] == 'China':
      ccode = 'CN'
    elif df.loc[i,'country'] == 'Netherlands':
      ccode = 'NL'

    k = df.index[(df['publication_year'] == df.loc[i,'publication_year']) & (df['country_code'] == ccode)][0]
    df.loc[k,'avg_value_usd'] = (df.loc[k,'avg_value_usd']*df.loc[k,'works'] +
                                 df.loc[i,'avg_value_usd']*df.loc[i,'works']) / (df.loc[k,'works'] + df.loc[i,'works'])
    row_drop.append(i)

df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
df.drop(row_drop,axis=0, inplace=True)
df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_physics',if_exists ='replace')


/tmp/ipython-input-1403392794.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(["The Netherlands"], ["Netherlands"], inplace=True)
/tmp/ipython-input-1403392794.py:22: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(project_id='insyspo',destination_table='userdb_luis_fabiano_borges.apcs_physics',if_exists ='replace')
100%|██████████| 1/1 [00:00<00:00, 21